In [16]:
#基于AdaBoost的营销响应预测
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score # 导入交叉验证算法
from sklearn.feature_selection import SelectPercentile, f_classif #导入特征选择方法库
from sklearn.ensemble import AdaBoostClassifier #导入集成算法
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import warnings

In [2]:
# 基本状态查看
def set_summary(df):
    '''
    查看数据集的记录数、维度数、前2条数据、描述性统计和数据类型
    :param df: 数据框
    :return: 无
    '''
    print ('Data Overview')
    print ('Records: {0}\tDimension{1}'.format(df.shape[0], (df.shape[1] - 1)))  # 打印数据集X形状
    print ('-' * 30)
    print (df.head(2))  # 打印前2条数据
    print ('-' * 30)
    print ('Data DESC')
    print (df.describe())  # 打印数据基本描述性信息
    print ('Data Dtypes')
    print (df.dtypes)  # 打印数据类型
    print ('-' * 60)


In [3]:
# 缺失值审查
def na_summary(df):
    '''
    查看数据集的缺失数据列、行记录数
    :param df: 数据框
    :return: 无
    '''
    na_cols = df.isnull().any(axis=0)  # 每一列是否具有缺失值
    print ('NA Cols:')
    print (na_cols)  # 查看具有缺失值的列
    print ('-' * 30)
    print ('valid records for each Cols:')
    print (df.count())  # 查看每一列有效值（非NA）的记录数
    print ('-' * 30)
    na_lines = df.isnull().any(axis=1)  # 查看每一行是否具有缺失值
    print ('Total number of NA lines is: {0}'.format(na_lines.sum()))  # 查看具有缺失值的行总记录数
    print ('-' * 30)

In [4]:
# 类样本均衡审查
def label_summary(df):
    '''
    查看每个类的样本量分布
    :param df: 数据框
    :return: 无
    '''
    print ('Labesl samples count:')
    print (df['value_level'].groupby(df['response']).count())  # 以response为分类汇总维度对value_level列计数统计
    print ('-' * 60)

In [5]:
# 数据预处理
# 变量类型转换
def type_con(df):
    '''
    转换目标列的数据为特定数据类型
    :param df: 数据框
    :return: 类型转换后的数据框
    '''
    var_list = {'edu': 'int32',
                'user_level': 'int32',
                'industry': 'int32',
                'value_level': 'int32',
                'act_level': 'int32',
                'sex': 'int32',
                'region': 'int32'
                }  # 字典：定义要转换的列及其数据类型
    for var, type in var_list.items():  # 循环读出列名和对应的数据类型
        df[var] = df[var].astype(type)  # 数据类型转换
    print ('Data Dtypes')
    print (df.dtypes)  # 打印数据类型
    print ('-' * 30)
    return df

In [6]:
# NA值替换
def na_replace(df):
    '''
    将数据集中的NA值使用自定义方法替换
    :param df: 数据框
    :return: NA值替换后的数据框
    '''
    na_rules = {'age': df['age'].mean(),
                'total_pageviews': df['total_pageviews'].mean(),
                'edu': df['edu'].median(),
                'edu_ages': df['edu_ages'].median(),
                'user_level': df['user_level'].median(),
                'industry': df['user_level'].median(),
                'act_level': df['act_level'].median(),
                'sex': df['sex'].median(),
                'red_money': df['red_money'].mean(),
                'region': df['region'].median()
                }  # 字典：定义各个列数据转换方法
    df = df.fillna(na_rules)  # 使用指定方法填充缺失值
    print ('Check NA exists:')
    print (df.isnull().any().sum())  # 查找是否还有缺失值
    print ('-' * 30)
    return df

In [7]:
# 标志转换
def symbol_con(df, enc_object=None, train=True):
    '''
    将分类和顺序变量转换为二值化的标志变量
    :param df: 数据框
    :param enc_object: sklearn的标志转换对象，训练阶段设置默认值为None；预测阶段使用从训练阶段获得的转换对象
    :param train: 是否为训练阶段的判断状态，训练阶段为True，预测阶段为False
    :return: 标志转换后的数据框、标志转换对象（如果是训练阶段）
    '''
    convert_cols = ['edu', 'user_level', 'industry', 'value_level', 'act_level', 'sex', 'region']  # 选择要做标志转换的列名
    df_con = df[convert_cols]  # 选择要做标志转换的数据
    df_org = df[['age', 'total_pageviews', 'edu_ages', 'blue_money', 'red_money', 'work_hours']].values  # 设置不作标志转换的列
    if train == True:  # 如果处于训练阶段
        enc = OneHotEncoder()  # 建立标志转换模型对象
        enc.fit(df_con)  # 训练模型
        df_con_new = enc.transform(df_con).toarray()  # 转换数据并输出为数组格式
        new_matrix = np.hstack((df_con_new, df_org))  # 将未转换的数据与转换后的数据合并
        return new_matrix, enc
    else:
        df_con_new = enc_object.transform(df_con).toarray()  # 使用训练阶段获得的转换对象转换数据并输出为数组格式
        new_matrix = np.hstack((df_con_new, df_org))  # 将未转换的数据与转换后的数据合并
        return new_matrix

In [8]:
# 获得最佳模型参数
def get_best_model(X, y):
    '''
    结合交叉检验得到不同参数下的分类模型结果
    :param X: 输入X（特征变量）
    :param y: 预测y（目标变量）
    :return: 特征选择模型对象
    '''
    transform = SelectPercentile(f_classif, percentile=50)  # 使用f_classif方法选择特征最明显的50%数量的特征
    model_adaboost = AdaBoostClassifier()  # 建立AdaBoostClassifier模型对象
    model_pipe = Pipeline(steps=[('ANOVA', transform), ('model_adaboost', model_adaboost)])  # 建立由特征选择和分类模型构成的“管道”对象
    cv = StratifiedKFold(5)  # 设置交叉检验次数
    n_estimators = [20, 50, 80, 100]  # 设置模型参数列表
    score_methods = ['accuracy', 'f1', 'precision', 'recall', 'roc_auc']  # 设置交叉检验指标
    mean_list = list()  # 建立空列表用于存放不同参数方法、交叉检验评估指标的均值列表
    std_list = list()  # 建立空列表用于存放不同参数方法、交叉检验评估指标的标准差列表
    for parameter in n_estimators:  # 循环读出每个参数值
        t1 = time.time()  # 记录训练开始的时间
        score_list = list()  # 建立空列表用于存放不同交叉检验下各个评估指标的详细数据
        print ('set parameters: %s' % parameter)  # 打印当前模型使用的参数
        for score_method in score_methods:  # 循环读出每个交叉检验指标
            model_pipe.set_params(model_adaboost__n_estimators=parameter)  # 通过“管道”设置分类模型参数
            score_tmp = cross_val_score(model_pipe, X, y, scoring=score_method, cv=cv)  # 使用交叉检验计算指定指标的得分
            score_list.append(score_tmp)  # 将交叉检验得分存储到列表
        score_matrix = pd.DataFrame(np.array(score_list), index=score_methods)  # 将交叉检验详细数据转换为矩阵
        score_mean = score_matrix.mean(axis=1).rename('mean')  # 计算每个评估指标的均值
        score_std = score_matrix.std(axis=1).rename('std')  # 计算每个评估指标的标准差
        score_pd = pd.concat([score_matrix, score_mean, score_std], axis=1)  # 将原始详细数据和均值、标准差合并
        mean_list.append(score_mean)  # 将每个参数得到的各指标均值追加到列表
        std_list.append(score_std)  # 将每个参数得到的各指标标准差追加到列表
        print (score_pd.round(2))  # 打印每个参数得到的交叉检验指标数据，只保留2位小数
        print ('-' * 60)
        t2 = time.time()  # 计算每个参数下算法用时
        tt = t2 - t1  # 计算时间间隔
        print ('time: %s' % str(tt))  # 打印时间间隔
    mean_matrix = np.array(mean_list).T  # 建立所有参数得到的交叉检验的均值矩阵
    std_matrix = np.array(std_list).T  # 建立所有参数得到的交叉检验的标准差矩阵
    mean_pd = pd.DataFrame(mean_matrix, index=score_methods, columns=n_estimators)  # 将均值矩阵转换为数据框
    std_pd = pd.DataFrame(std_matrix, index=score_methods, columns=n_estimators)  # 将均值标准差转换为数据框
    print ('Mean values for each parameter:')
    print (mean_pd)  # 打印输出均值矩阵
    print ('Std values for each parameter:')
    print (std_pd)  # 打印输出标准差矩阵
    print ('-' * 60)
    return transform

In [9]:
warnings.filterwarnings('ignore')
# 数据应用
# 加载数据集
raw_data = pd.read_excel('order.xlsx', sheetname=0)  # 读出Excel的第一个sheet
X = raw_data.drop('response', axis=1)  # 分割X
y = raw_data['response']  # 分割y

In [10]:
# 数据审查和预处理
set_summary(raw_data)  # 基本状态查看
na_summary(raw_data)  # 缺失值审查
label_summary(raw_data)  # 类样本均衡均衡审查
X_t1 = na_replace(X)  # 替换缺失值
X_t2 = type_con(X_t1)  # 数据类型转换
X_new, enc = symbol_con(X_t2, enc_object=None, train=True)  # 将分类和顺序数据转换为标志

Data Overview
Records: 39999	Dimension13
------------------------------
    age  total_pageviews  edu  edu_ages  user_level  industry  value_level  \
0  39.0          77516.0  1.0      13.0         1.0       1.0            1   
1  50.0          83311.0  1.0      13.0         2.0       2.0            2   

   act_level  sex  blue_money  red_money  work_hours  region  response  
0        1.0  1.0        2174        0.0          40     1.0         0  
1        1.0  1.0           0        0.0          13     1.0         0  
------------------------------
Data DESC
                age  total_pageviews           edu      edu_ages  \
count  39998.000000     3.999800e+04  39998.000000  39998.000000   
mean      38.589654     1.895136e+05      2.511626     10.076754   
std       13.663490     1.053109e+05      1.638110      2.573384   
min       17.000000     1.228500e+04      1.000000      1.000000   
25%       28.000000     1.175282e+05      2.000000      9.000000   
50%       37.000000     1

In [11]:
# 分类模型训练
transform = get_best_model(X_new, y)  # 获得最佳分类模型参数信息
transform.fit(X_new, y)  # 应用特征选择对象选择要参与建模的特征变量
X_final = transform.transform(X_new)  # 获得具有显著性特征的特征变量
final_model = AdaBoostClassifier(n_estimators=100)  # 从打印的参数均值和标准差信息中确定参数并建立分类模型对象
final_model.fit(X_final, y)  # 训练模型  

set parameters: 20
              0     1     2     3     4  mean   std
accuracy   0.85  0.85  0.86  0.86  0.86  0.85  0.01
f1         0.66  0.64  0.66  0.66  0.66  0.66  0.01
precision  0.72  0.74  0.76  0.78  0.77  0.75  0.02
recall     0.60  0.57  0.59  0.58  0.57  0.58  0.01
roc_auc    0.91  0.90  0.91  0.91  0.91  0.91  0.00
------------------------------------------------------------
time: 9.742232084274292
set parameters: 50
              0     1     2     3     4  mean   std
accuracy   0.86  0.86  0.86  0.87  0.86  0.86  0.00
f1         0.66  0.66  0.67  0.69  0.68  0.67  0.01
precision  0.75  0.76  0.77  0.78  0.77  0.76  0.01
recall     0.59  0.58  0.59  0.62  0.61  0.60  0.02
roc_auc    0.91  0.91  0.91  0.92  0.92  0.91  0.00
------------------------------------------------------------
time: 20.814851999282837
set parameters: 80
              0     1     2     3     4  mean   std
accuracy   0.86  0.86  0.86  0.87  0.86  0.86  0.00
f1         0.67  0.67  0.68  0.70  0.68  0.6

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None)

In [12]:
# 新数据集做预测
new_data = pd.read_excel('order.xlsx', sheetname=1)  # 读取要预测的数据集
final_reponse = new_data['final_response']  # 获取最终的目标变量值
new_data = new_data.drop('final_response', axis=1)  # 获得预测的输入变量X
set_summary(new_data)  # 基本状态查看
na_summary(new_data)  # 缺失值审查
new_X_t1 = na_replace(new_data)  # 替换缺失值
new_X_t2 = type_con(new_X_t1)  # 数据类型转换
new_X_t3 = symbol_con(new_X_t2, enc_object=enc, train=False)  # 将分类和顺序数据转换为标志
new_X_final = transform.transform(new_X_t3)  # 对数据集做特征选择

Data Overview
Records: 8843	Dimension12
------------------------------
   age  total_pageviews  edu  edu_ages  user_level  industry  value_level  \
0   61           243019   10         1         2.0       7.0            2   
1   33           215596    4         5         2.0       7.0            2   

   act_level  sex  blue_money  red_money  work_hours  region  
0          1    1           0          0          40     1.0  
1          5    1           0          0          40     6.0  
------------------------------
Data DESC
               age  total_pageviews          edu     edu_ages   user_level  \
count  8843.000000     8.843000e+03  8843.000000  8843.000000  8841.000000   
mean     38.884428     1.903636e+05     2.492141    10.083795     2.070015   
std      13.917154     1.069146e+05     1.603766     2.560132     1.241608   
min      17.000000     1.349200e+04     1.000000     1.000000     1.000000   
25%      28.000000     1.177010e+05     2.000000     9.000000     1.000000   

In [13]:
# 输出预测值以及预测概率
predict_labels = pd.DataFrame(final_model.predict(new_X_final), columns=['labels'])  # 获得预测标签
predict_labels_pro = pd.DataFrame(final_model.predict_proba(new_X_final), columns=['pro1', 'pro2'])  # 获得预测概率
predict_pd = pd.concat((new_data, predict_labels, predict_labels_pro), axis=1)  # 将预测标签、预测数据和原始数据X合并
print ('Predict info')
print (predict_pd.head(2))  # 打印前2条结果
print ('-' * 60)

Predict info
   age  total_pageviews  edu  edu_ages  user_level  industry  value_level  \
0   61           243019   10         1         2.0       7.0            2   
1   33           215596    4         5         2.0       7.0            2   

   act_level  sex  blue_money  red_money  work_hours  region  labels  \
0          1    1           0          0          40     1.0       0   
1          5    1           0          0          40     6.0       0   

       pro1      pro2  
0  0.504053  0.495947  
1  0.507486  0.492514  
------------------------------------------------------------


In [14]:
# 将预测结果写入Excel
writer = pd.ExcelWriter('order_predict_result.xlsx')  # 创建写入文件对象
predict_pd.to_excel(writer, 'Sheet1')  # 将数据写入sheet1
writer.save()  # 保存文件

In [15]:
# 后续--与实际效果的比较
print ('final accuracy: {0}'.format(accuracy_score(final_reponse, predict_labels)))

final accuracy: 0.8624901051679295
